# **College Student Placement Factors Dataset** #

#### Credits to `SAHIL ISLAM007` for the dataset. ####

Source: https://www.kaggle.com/datasets/sahilislam007/college-student-placement-factors-dataset?resource=download


## Libraries ##

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import zscore

## Dataset ##

In [ ]:
path = r'Replace with data path'
data = pd.read_csv(path)

In [92]:
print(data.describe())
#print(data.head(5))

                 IQ  Prev_Sem_Result          CGPA  Academic_Performance  \
count  10000.000000     10000.000000  10000.000000          10000.000000   
mean      99.471800         7.535673      7.532379              5.546400   
std       15.053101         1.447519      1.470141              2.873477   
min       41.000000         5.000000      4.540000              1.000000   
25%       89.000000         6.290000      6.290000              3.000000   
50%       99.000000         7.560000      7.550000              6.000000   
75%      110.000000         8.790000      8.770000              8.000000   
max      158.000000        10.000000     10.460000             10.000000   

       Extra_Curricular_Score  Communication_Skills  Projects_Completed  
count            10000.000000          10000.000000        10000.000000  
mean                 4.970900              5.561800            2.513400  
std                  3.160103              2.900866            1.715959  
min                

### Known Fields ###
- **College_ID**
    - Unique ID of colleges.
- **IQ**
    - Student IQ score (normally distributed around 100.)
- **Prev_Sem_Result**
    -  Grade point average of previous semester (range of 5.0 to 10.0).
- **CGPA**
    - Cumulative Grde Point Average  (range of ~5.0 to 10.0)
- **Academic_Performance**
    - Scale (1 to 10) for annual academic rating.
- **Internship Experience**
    - Boolean ('Yes'/'No') for student past internship experience.
- **Communication_Skills**
    - Scale (1 to 10) for communication ability rating.
- **Projects_Completed**
    - Count (0 to 5) of projects completed.
- **Placement**
    - Boolean ('Yes'/'No') of whether student got a placement.

## Objectives (Current) ##
- Clean/normalize/adjust dataset for data analysis if needed.
- Identify preliminary (linear) relationships between data using a correlation matrix with p-values for significance.
- Identify key factors and their importance that influence placement using factor analysis (or other) methods.
- Consider feature engineer a ``Student_Score`` using factors (and weightings if needed) for determining placement.
- Test ``Student_Score`` for suitability using a decision tree.
- If suitable, identify key thresholds that can be used for automating placement decisions using ``Student_Score``. Consider methods such as logistic regression or random forest.
- Train a model to categorise whether a student will get a placement or not. Consider ensemble learning methods for improved accuracy.
- Test the model for suitability.

## Cleaning Dataset ##

In [79]:
empty_columns = data.columns[data.isnull().any()].to_list()
print("Columns with empty values: ", empty_columns)

Columns with empty values:  []


In [80]:
dupes = data[data.duplicated()]
print("Duplicate values: ", dupes.shape[0])

Duplicate values:  0


#### Outlier Detection w/ z-Score ####
Outliers should only be possible in quantitative fields.

In [123]:
 #Possible fields: 'IQ', 'Prev_Sem_Result', 'CGPA', 'Academic_Performance', 'Communication_Skills', 'Projects_Completed'

#Parameters
outlier_std = 3 #Default is 3.
quantitative_fields = ['IQ', 'Prev_Sem_Result', 'CGPA', 'Academic_Performance', 'Communication_Skills', 'Projects_Completed']


#### Note: ####
Function `find_outliers` will re-define z-scores when called. <br>
Therefore updating the dataset AND calling the function after, can cause new outliers to re-emerge based on updated dataset.

In [124]:
#Return relevant outlier values.
def find_outliers(param_field):
    data_z = pd.DataFrame(zscore(data[param_field], nan_policy='omit'), columns=[param_field], index=data.index)
    outlier_mask = (abs(data_z) > outlier_std).any(axis=1)
    outliers = data[outlier_mask]
    
    return outlier_mask, outliers
    
mask, outliers = find_outliers(quantitative_fields)

print(outliers)

     College_ID   IQ  Prev_Sem_Result  CGPA  Academic_Performance  \
209     CLG0073  157             8.33  8.53                     4   
262     CLG0004   51             7.26  7.43                     1   
478     CLG0047  146             9.72  9.44                     8   
1615    CLG0037  147             8.34  7.90                     6   
1957    CLG0037  147             7.58  7.14                     4   
2024    CLG0044   54             9.04  8.82                     4   
2305    CLG0047  146             5.81  5.37                     8   
2895    CLG0056  158             6.07  5.89                     8   
3716    CLG0033  148             9.37  9.08                     1   
3982    CLG0037  147             5.15  5.32                     9   
4198    CLG0071   54             9.07  9.27                     7   
4725    CLG0017   52             8.92  9.36                     8   
4779    CLG0057   52             7.56  7.36                     9   
4870    CLG0100  146             9

In [125]:
#Determine which columns have outliers above the specified standard deviation, and how many.
for field in quantitative_fields:
    _ , outliers = find_outliers(field)
    print(f"Number of outliers in {field}: {outliers.shape[0]}")

Number of outliers in IQ: 28
Number of outliers in Prev_Sem_Result: 0
Number of outliers in CGPA: 0
Number of outliers in Academic_Performance: 0
Number of outliers in Communication_Skills: 0
Number of outliers in Projects_Completed: 0


#### Replace Outliers ####

In [ ]:
IQ_median = data['IQ'].median()
print("Median of IQ: ", IQ_median, '\n')

IQ_outlier_mask, _ = find_outliers(['IQ'])
IQ_outlier_indices = data[IQ_outlier_mask].index.tolist()

#Replace outliers of IQ values with median.
data.loc[IQ_outlier_mask, 'IQ'] = IQ_median

#Check that the outlier values have been replaced with median (99 in this case), using indexes from mask.
print("Replaced values:\n", data.loc[IQ_outlier_indices, 'IQ'])


Median of IQ:  99.0 

Replaced values:
 209     99
262     99
478     99
1615    99
1957    99
2024    99
2305    99
2895    99
3716    99
3982    99
4198    99
4725    99
4779    99
4870    99
4997    99
5796    99
6891    99
7140    99
7223    99
7305    99
7611    99
7677    99
8024    99
8180    99
8200    99
8248    99
8726    99
9770    99
Name: IQ, dtype: int64
